In [1]:
import sys
print(sys.executable)
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install faster-whisper openai requests tqdm transformers pillow opencv-python torch torchvision torchaudio
!{sys.executable} -m pip install ipywidgets

/opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/bin/python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchaudio]2 [torchaudio]


In [2]:
import os
from pathlib import Path
import json
import time
import requests

#openai for LLM functionality
OPENAI_API_KEY = ""
#assembly for video to text script
ASSEMBLYAI_API_KEY = ""
#google cloud for video analysis (also try Amazon Rekognition, Microsoft Azure Video Indexer, Clarifai)
GCP_API_KEY = ""

VIDEO_PATH = "good_place_clip.mp4"


In [3]:
def transcribe_to_text(video_path: str) -> dict:
    headers = {"authorization": ASSEMBLYAI_API_KEY}
    # uplkoad video
    upload_url = "https://api.assemblyai.com/v2/upload"
    def read_file(filename, chunk_size=5242880):
        with open(filename, 'rb') as f:
            while True:
                data = f.read(chunk_size)
                if not data:
                    break
                yield data
    up_resp = requests.post(upload_url, headers=headers, data=read_file(video_path))
    audio_url = up_resp.json()['upload_url']

    # transcribe
    transcribe_url = "https://api.assemblyai.com/v2/transcript"
    transcribe_req = {"audio_url": audio_url, 
                      "speaker_labels": True,
                      "iab_categories": True, # topics
                      "entity_detection": True,
                      "sentiment_analysis": True,
                      "auto_chapters": True}
    t_resp = requests.post(transcribe_url, headers=headers, json=transcribe_req)
    transcript_id = t_resp.json()['id']

    # keep polling until complete
    poll_url = f"{transcribe_url}/{transcript_id}"
    while True:
        poll_resp = requests.get(poll_url, headers=headers)
        status = poll_resp.json()['status']
        if status == 'completed':
            return poll_resp.json()
        elif status == 'error':
            raise RuntimeError(poll_resp.json()['error'])
        time.sleep(5)


In [12]:
print(os.getcwd())
transcript_result = transcribe_to_text(VIDEO_PATH)
transcript_text = transcript_result.get("text", "")

/Users/estellekim/Projects/5810_video_analysis


In [13]:
print("Text:")
print(transcript_result.get("text", "")) 

print("Entities:")
for e in transcript_result.get("entities", []):
    print(f"{e['entity_type']}: {e['text']}")

print("Categories:")
if "iab_categories_result" in transcript_result:
    summary = transcript_result["iab_categories_result"].get("summary", {})
    for name in summary.items():
        print(name)

Text:
Was I ever good at being sad? Partly because my mom straight up told me not to be. But this is sad, man. You got a John Locke quote or a piece of Kantian wisdom you can throw at me? Those guys were more focused on rules and regulations. For spiritual stuff, you gotta turn to the east. I'll take anything you got. Hit me. Picture a wave in the ocean. You can see it. Measure it. Its height, the way the sunlight refracts when it passes through. And it's there and you can see it. You know what it is? It's a wave. And then it crashes on the shore. And it's gone. But the water is still there. The wave was just a different way for the water to be for a little while. That's one conception of death for a Buddhist. The wave returns to the ocean where it came from and where it's supposed to be. Not bad. Buddhists, not bad. None of this is bad. I need you to do me one last favor. Say goodbye to me now and leave before I wake up. Sam. You can sit on that bench as long as you'd like. And whenev

In [6]:
from faster_whisper import WhisperModel

model = WhisperModel("small")
segments, info = model.transcribe(VIDEO_PATH)
transcript = [{"start": s.start, "end": s.end, "text": s.text} for s in segments]

[2025-10-03 00:00:56.940] [ctranslate2] [thread 5310330] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


In [7]:
transcript_text_only = ''.join([s["text"] for s in transcript])
print(transcript_text_only)
print(transcript)

 What was ever good at being sad? Probably because my mom straight up told me not to be. But this is sad, man. You got a John Locke quote, or a piece of county and wisdom you can throw at me? Those guys were more focused on rules and regulations. For spiritual stuff, you got to turn to the east. I'll take anything you got. Hit me. Picture a wave in the ocean. You can see it, measure it, its height, the way the sunlight refracts when it passes through, and it's there, and you can see it. You know what it is. It's a wave. And then it crashes on the shore, and it's gone, but the water is still there. The wave was just a different wave for the water to be for a little while. That's one conception of death for a Buddhist. The wave returns to the ocean. Where it came from. And where it's supposed to be. Not bad, Buddhists. Not bad. None of this is bad. I need you to do me one last favor. Say goodbye to me now, and leave before I wake up. You can sit on that bench as long as you'd like, and w

In [14]:
# take frames every 1 second of the video to put into image analysis

import cv2

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
frames = []
frame_id = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_id % int(fps) == 0:  # 1 fps
        frames.append(frame)
    frame_id += 1
cap.release()

In [9]:
# print(frames)

In [10]:
# analyze and capture each frame

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image

# https://huggingface.co/Salesforce/blip-image-captioning-base
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

captions = []
for f in frames:
    image = Image.fromarray(cv2.cvtColor(f, cv2.COLOR_BGR2RGB))
    inputs = processor(images=image, return_tensors="pt")
    out = model.generate(**inputs)
    captions.append(processor.decode(out[0], skip_special_tokens=True))

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [11]:
print(captions)

['a black background with a white and red flower', 'a person sitting on a balcony watching the sunset', 'a person sitting on a couch watching a sunset', 'a couple sitting on a balcony watching the sunset', 'a couple sitting on a balcony watching the sunset', 'a couple sitting on a couch watching the sunset', 'a woman sitting on a couch', 'a woman sitting on a couch with a man', 'a woman sitting on a couch next to a man', 'a woman sitting on a couch next to a man', 'a woman sitting on a couch', 'a woman sitting on a couch', 'a woman sitting on a couch next to a man', 'a woman sitting on a couch', 'watch this gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi', 'a woman sitting on a couch next to a man', 'watch this video of a $ $ $ from the movie', 'watch this gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi', 'watch this gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi gi', 'a woman sitting on a couch next to a man', 'a man and woman sitting on a couch', 'a man and woman sitt

In [ ]:
# aggregate frame descrptions with text transcript and previous assembly results

In [ ]:
# output as organized text